In [18]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *

from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

In [19]:
def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print(full)
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False

#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 30
    a = np.random.randint(1, max, size=3)
    b = np.random.randint(1, max, size=3)
    c = np.random.randint(1, max, size=3)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 3, 3)) * 1.0
    return (tens, a,b,c)

In [20]:
def check(t, p = False):
    p000 = t[0][0][0]
    p001 = t[1][0][0]
    p002 = t[2][0][0]
    p100 = t[0][1][0]
    p101 = t[1][1][0]
    p102 = t[2][1][0]
    p010 = t[0][0][1]
    p011 = t[1][0][1]
    p012 = t[2][0][1]
    p110 = t[0][1][1]
    p111 = t[1][1][1]
    p112 = t[2][1][1]
    p210 = t[0][2][1]
    p211 = t[1][1][2]
    p212 = t[2][2][1]
    p220 = t[0][2][2]
    p221 = t[1][2][2]
    p222 = t[2][2][2]
    p120 = t[0][1][2]
    p121 = t[1][1][2]
    p122 = t[2][1][2]
    p200 = t[0][2][0]
    p201 = t[1][2][0]
    p202 = t[2][2][0]
    p020 = t[0][0][2]
    p021 = t[1][0][2]
    p022 = t[2][0][2]
    

def check_req(p):
    d1 = Matrix(p[0])
    d2 = Matrix(p[1])
    d3 = Matrix(p[2])
    v1 = Matrix(p[:,:,0])
    v2 = Matrix(p[:,:,1])
    v3 = Matrix(p[:,:,2])
    h1 = Matrix(p[:,0])
    h2 = Matrix(p[:,1])
    h3 = Matrix(p[:,2])
    zero = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    #d1a = d1.adjugate()
    d2a = d2.adjugate()
    #d3a = d3.adjugate()
    #v1a = v1.adjugate()
    #v2a = v2.adjugate()
    #v3a = v3.adjugate()
    #h1a = h1.adjugate()
    #h2a = h2.adjugate()
    #h3a = h3.adjugate()
    a1 = d1 * d2a * d3 - d3* d2a * d1
    #a2 = d2 * d3a * d1 - d1* d3a * d2
    #a3 = d3 * d1a * d2 - d2* d1a * d3
    #b1 = v1 * v2a * v3 - v3* v2a * v1 
    #b2 = v2 * v3a * v1 - v1* v3a * v2 
    #b3 = v3 * v1a * v2 - v2* v1a * v3 
    #c1 = h1 * h2a * h3 - h3* h2a * h1
    #c2 = h2 * h3a * h1 - h1* h3a * h2 
    #c3 = h3 * h1a * h2 - h2* h1a * h3 
    #print(a1,a2,a3)
    #print(a,b,c)
    #a = check(a1) and check(a2) and check(a3)
    #b = check(b1) and check(b2) and check(b3)
    #c = check(c1) and check(c2) and check(c3)
    return check(a1)
    
def check(a):
    return max(abs(a)) < 5


In [ ]:
wrong = []

In [22]:
res = {True:0, False:0}

for i in range(1000000):
    t1 = low_tensor()
    t2 = low_tensor()
    t3 = low_tensor()
    t4 = low_tensor()
    v = np.random.randint(0,100)
    t = t1[0] + t2[0] + t3[0] + t4[0]
    if v%2 == 0:
        t = np.random.randint(0,100000,size = (3,3,3))
    key = check_req(t)
    res[key] = res[key] + 1
    if key and not check_rank(t,3, tol = 0.01):
        print(t,v)
        wrong.append((t,t1,t2,t3,t4,v))
        print(check_rank(t,3))
print(res)

[[[ 8464. 11008.  7824.]
  [14973. 13101. 12702.]
  [19205. 18605. 16614.]]

 [[20074. 25890. 17454.]
  [23667. 22521. 17680.]
  [33704. 35466. 26407.]]

 [[16954. 20622. 15542.]
  [26199. 23718. 22799.]
  [34676. 34029. 30570.]]] 45
False
[[[15126. 12576. 28395.]
  [18448. 17040. 30385.]
  [16787. 14808. 29390.]]

 [[ 9620.  9090. 15738.]
  [12252. 12438. 17582.]
  [10936. 10764. 16660.]]

 [[ 5820.  5478. 12762.]
  [ 9362. 10494. 15414.]
  [ 7591.  7986. 14088.]]] 91
False
{True: 5, False: 999995}


In [23]:
print(wrong)

[(array([[[ 8464., 11008.,  7824.],
        [14973., 13101., 12702.],
        [19205., 18605., 16614.]],

       [[20074., 25890., 17454.],
        [23667., 22521., 17680.],
        [33704., 35466., 26407.]],

       [[16954., 20622., 15542.],
        [26199., 23718., 22799.],
        [34676., 34029., 30570.]]]), (array([[[ 3024.,  2800.,  2240.],
        [ 2835.,  2625.,  2100.],
        [ 4347.,  4025.,  3220.]],

       [[11664., 10800.,  8640.],
        [10935., 10125.,  8100.],
        [16767., 15525., 12420.]],

       [[10368.,  9600.,  7680.],
        [ 9720.,  9000.,  7200.],
        [14904., 13800., 11040.]]]), array([ 7, 27, 24]), array([16, 15, 23]), array([27, 25, 20])), (array([[[2520., 1008.,  168.],
        [2880., 1152.,  192.],
        [4140., 1656.,  276.]],

       [[5250., 2100.,  350.],
        [6000., 2400.,  400.],
        [8625., 3450.,  575.]],

       [[2310.,  924.,  154.],
        [2640., 1056.,  176.],
        [3795., 1518.,  253.]]]), array([12, 25, 11]),

In [178]:
result = {True:0, False:0}
for i in range(10000):
    t = low_tensor()[0] + low_tensor()[0] + low_tensor()[0]
    key = min_det(t)
    result[key] = result[key] + 1
print(result)

{True: 9980, False: 20}


In [174]:
t= wrong[1][0]
min_det(t)

False

In [157]:
def min_det(t):
    #t = tl.tensor(low_tensor()[0] + low_tensor()[0] + low_tensor()[0] + low_tensor()[0])
    P1s = Matrix(t[:,0] + t[:,1] + t[:,2]).transpose()
    P2s = Matrix(t[:,:,0] + t[:,:,1] + t[:,:,2]).transpose()
    P3s = Matrix(t[0] + t[1] + t[2]).transpose()
    P1a = P1s.adjugate()
    dP1s = det(P1s)
    A1 = P2s * P1a * t[0].transpose() * dP1s
    A2 = P2s * P1a  * t[1].transpose() * dP1s
    A3 = P2s * P1a * t[2].transpose() * dP1s
    a = check_prin(Matrix(A1)) and check_prin(Matrix(A2)) and check_prin(Matrix(A3))
    B1 = t[:,:,0].transpose() * P1a * P3s * dP1s
    B2 = t[:,:,1].transpose() * P1a * P3s * dP1s
    B3 = t[:,:,2].transpose() * P1a * P3s * dP1s
    b = check_prin(B1) and check_prin(B2) and check_prin(B3)
    C1 = t[:,0].transpose() * P2s.adjugate() * P3s.transpose() * det(P2s)
    C2 = t[:,1].transpose() * P2s.adjugate() * P3s.transpose() * det(P2s)
    C3 = t[:,2].transpose() * P2s.adjugate() * P3s.transpose() * det(P2s)
    c = check_prin(C1) and check_prin(C2) and check_prin(C3)
    return a and b and c



In [171]:
def check_prin(A):
    a1 = A[0,0] > 0
    a2 = A.minor(2,2) > 0
    A = A/1e+20
    a3 = det(A) > 0
    #print(A[0,0], A.minor(2,2), det(A))
    return a1 and a2 and a3

In [120]:
K1 = A1/1e+15
K1s = P1s
K2s = P2s
K3s = P3s
B1 = K2s * P1s.adjugate() * t[0].transpose() * det(P1s)
det(K1)

1.38551028929870e+32

In [121]:
det(A1)/(1e+15)**3

1.38551028929868e+32